In [ ]:
from pyspark.sql import SparkSession
 # Yazıları Vectore çevir
from pyspark.ml.feature import VectorAssembler
# Logistic Regression yerine başka bir algoritma uygulanabilir
from pyspark.ml.classification import GBTClassifier # Logistic Regression import
from pyspark.ml.evaluation import BinaryClassificationEvaluator # Sonuç binary olacağı için

spark = SparkSession.builder.appName('Churn Classifier').getOrCreate()

data=spark.read.csv('churn.csv', header=True, inferSchema=True)
predictors = data.columns[:-1]
assembler = VectorAssembler(inputCols=predictors, outputCol='features')
data = assembler.transform(data).select('features', 'Outcome')

train_data, test_data = data.randomSplit([0.80,0.2], seed=42) # 42 evrenin anlamı

gbt = GBTClassifier(labelCol='Outcome', featuresCol='features')

model=gbt.fit(train_data)

predictions=model.transform(test_data)

evaluator=BinaryClassificationEvaluator(labelCol='Outcome')
# Başka algoritma uygulanabilir accuracy yerine r2 kullanılabilir
accuracy=evaluator.evaluate(predictions)
print(f"Accuracy", {accuracy})

spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Spark session oluşturulması
spark = SparkSession.builder.appName('Churn Classifier').getOrCreate()

# Veriyi okuma
data = spark.read.csv('churn.csv', header=True, inferSchema=True)

# Kategorik değişkenleri sayısal verilere dönüştürme işlemi gerekebilir.
# Eğer string tipinde sütunlar varsa StringIndexer kullanılmalı.

# Predictor sütunlarının belirlenmesi (son sütunu Outcome olarak varsayıyoruz)
predictors = data.columns[:-1]

# Veriyi birleştirerek özellik vektörü oluşturma
assembler = VectorAssembler(inputCols=predictors, outputCol='features')
data = assembler.transform(data).select('features', data.columns[-1].alias('Outcome')) # sonu değişmiş

# Veriyi eğitim ve test olarak ayırma
train_data, test_data = data.randomSplit([0.80, 0.20], seed=42)  # 42 sabit bir rastgelelik sağlar

# Modelin oluşturulması
gbt = GBTClassifier(labelCol='Outcome', featuresCol='features')

# Modelin eğitilmesi
model = gbt.fit(train_data)

# Test veri seti üzerinde tahmin yapma
predictions = model.transform(test_data)

# Modeli değerlendirme (Accuracy hesaplaması)
evaluator = BinaryClassificationEvaluator(labelCol='Outcome')
accuracy = evaluator.evaluate(predictions)

# Sonuçları yazdırma
print(f"Accuracy: {accuracy}")

# Spark oturumunu sonlandırma
spark.stop()
